Importing all the necassary libraries

In [73]:
import numpy as np
import pandas as pd
import os
import math

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Creating a confusion matrix class that is used to evaluate the trained model after testing it on the test set

In [74]:
class confusion_matrix:
  def __init__(self):
    self.true_Real = 0
    self.true_Fake = 0
    self.false_Real = 0
    self.false_Fake = 0
    self.total = 0
    self.right = 0
  
  def update_matrix(self , actual , predicted ):
    self.total += 1
    if actual == predicted:
      self.right += 1
      if actual == 'Positive':
        self.true_Real += 1
      else:
        self.true_Fake += 1

    if actual != predicted:
      if actual == 'Positive':
        self.false_Real += 1
      else:
        self.false_Fake += 1
  
  def results(self):
    p = self.true_Real/(self.true_Real + self.false_Real )
    r = self.true_Real/(self.true_Real + self.false_Fake )
    print( "Accuracy : " , self.right/(self.total )  * 100)
    print( "Precision: " , self.true_Real/(self.true_Real + self.false_Real ) * 100 )
    print( "Recall   : " , self.true_Real/(self.true_Real + self.false_Fake )* 100 )
    print( "F1 score : " , 2* ((p*r)/(p+r)) * 100 )

Reading data from the given training data set and inserting it into their respective classes

In [76]:
classes = ['Negative' , 'Positive']

Train = {}
list_of_df = []

for file_name in os.listdir(r'drive/My Drive/FYP/Twitter Data/'):
  data = pd.read_csv(r'drive/My Drive/FYP/Twitter Data/' + file_name , encoding='latin-1' , names = ['sentiment', 'id1' , 'Date' , 'query' , 'name', 'text'])
  list_of_df.append(data)

data = pd.concat(list_of_df, ignore_index=True)

In [77]:
# Drop columns unnecassry
data = data.drop(['id1' , 'Date' , 'query' , 'name'] , axis=1)

In [78]:
# naming sentiments
data["sentiment"].replace({0: "Negative", 4: "Positive"}, inplace=True)

In [79]:
# Shuffle the dataset
data = data.sample(frac=1).reset_index(drop=True)

In [80]:
# !pip install -q -U watermark
# !pip install -qq transformers

In [81]:
%reload_ext watermark
%watermark -v -p numpy,pandas,torch,transformers

Python implementation: CPython
Python version       : 3.7.12
IPython version      : 5.5.0

numpy       : 1.19.5
pandas      : 1.1.5
torch       : 1.10.0+cu111
transformers: 4.14.1



In [84]:
import transformers
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup
import torch

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [85]:
Train_df, Test_df = train_test_split(data, test_size=0.1, random_state=RANDOM_SEED)
Val_df, Test_df = train_test_split(Test_df, test_size=0.5, random_state=RANDOM_SEED)

In [86]:
# Dividing Data into train and test Sets

# Split_Ratio = 0.8

# Train_df = data.iloc[: int(len(data)*Split_Ratio) , :].reset_index(drop=True)
# Test_df = data.iloc[int((len(data)*Split_Ratio)) + 1: , :].reset_index(drop=True)

In [87]:
Train_df

,sentiment,text
20363,Positive,heading to cinema later.
1378676,Positive,@nrthwstgal The store didn't have beef in the ...
1255287,Positive,Off work early. Yay
12577,Positive,@saraxoxoxo wait waaa are u getting a new phon...
600281,Negative,I hope Shannons divorce isn't 2 brutal.... why...
...,...,...
259178,Positive,Going home.. Ill see all my truestar @ 6ish
1414414,Positive,Is watching tv with danica and delania
131932,Negative,@RogueReverend One of the comments said &quot;...
671155,Positive,is waiting for the trainnnnn


In [88]:
Train = dict.fromkeys(classes,[])
Test = dict.fromkeys(classes,[])

for c in classes:
  Train[c] = Train_df.loc[Train_df['sentiment'] == c ]['text'].tolist()
  Test[c] = Test_df.loc[Test_df['sentiment'] == c]['text'].tolist()

Reading the given stop words 

In [89]:
#### Read the stop words
# f = open( r'drive/My Drive/NLP/Assignment 5/data/stopwords-ur.txt'  ,'rt', encoding="utf-8-sig" )
# stop_words = f.read().split('\n')

A function that takes the training data set and makes a vocabulary of unique words. It also takes in a bool that determines whether it should include stopwords or not

In [90]:
def makeVocabulary(T , stopwords = False):
  V = set()
  for c in T:
    for text in T[c]:
      words = text.split()
      for word in words:
        if stopwords == True and word in stop_words:
          continue
        V.add(word)
  V = list(V)
  return V

Function that given a set of classes and train dataset, Trains a conditionalProbability model for fake news classification. It also takes in two bools. Bool 'booleanNB' determines whether the boolean Naive Bayes algorithm will be used to train the data set and boolean 'stopwords' determine whether the stopwords are to be considered while training

In [91]:
def TrainMultinomialNB(C, T , booleanNB = False , stopwords = False ):
  V = makeVocabulary(T , stopwords)
  Priors = {}
  condProb = {}

  sum_of_priors = 0

  for c in C:
    sum_of_priors += len(T[c])
  
  for c in C:
    Priors[c] = len(T[c])/sum_of_priors

    doc = []
    for text in T[c]:
      templist = text.split()

      if booleanNB == True:
        doc += list(set(text.split()))
      else:
        doc += text.split()

    count_words = dict.fromkeys( V , 0 )
    for word in doc:
      if word in count_words.keys():
        if stopwords == True and word in stop_words:
          continue
        count_words[word] += 1

    
    for word in V:
      condProb[word , c] = ( count_words[word] + 1 ) / ( len(doc) + len(V) ) 

  return V , Priors , condProb

A function that takes in the vocabulary, the classes, the condtitional probabilty model and the test text and determines in which class the test text belongs. It also takes in a bool to specify whether it should consider stopwords or not 

In [92]:
def ApplyMultinomialNB(C, V, prior, condprob, t , stopwords = False):
  words_from_text = t.split()
  score = dict.fromkeys(C , 0)
  
  for c in C:
    score[c] = math.log(prior[c])
    for word in words_from_text:
      if stopwords == True and word in stop_words:
        continue
      if (word , c) in condprob.keys():
        score[c] += math.log(condprob[word , c])

  if (score['Positive'] > score['Negative']):
    return 'Positive'
  else:
    return 'Negative'

# Train and test the **Naive Baise Model with Stop Words** on the test set

In [93]:
### Training Naive Baise Model with Stop Words
vocab , priors , cond_Prob = TrainMultinomialNB(classes , Train , booleanNB = False , stopwords = False)

In [94]:
results_of_NB_withStopWords = confusion_matrix()
for c in classes:
  for text in Test[c]:
    result = ApplyMultinomialNB(classes, vocab, priors, cond_Prob, text , stopwords = False)
    results_of_NB_withStopWords.update_matrix(c , result)
  
results_of_NB_withStopWords.results()

Accuracy :  78.34402069974125
Precision:  73.19006781013164
Recall   :  81.71115255086421
F1 score :  77.2162386081193
